In [ ]:
# -*- coding: utf-8 -*-
"""retropropagación_basica.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1onSdgqmJ9yOWqr_uHohU4lOPGsdInPPc

Programa red neural movie review realizado Venkataramanan A.
Editado 21-09-2023 para la clase de TSAP, UPIIT
"""

from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

"""**El siguiente código descarga el conjunto de datos IMDB a su máquina (o utiliza una copia en caché si ya lo ha descargado):**"""

imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

"""**El argumento num_words=10000 mantiene las 10.000 palabras más frecuentes en los datos de entrenamiento. Las palabras poco frecuentes se descartan para mantener un tamaño manejable de los datos.**"""

print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

"""**El texto de las reseñas se ha convertido en números enteros, donde cada número entero representa una palabra específica en un diccionario. Este es el aspecto de la primera reseña:**"""

print(train_data[0])

"""*Las reseñas de las películas pueden tener distintas longitudes. El código siguiente muestra el número de palabras de la primera y la segunda crítica. Dado que las entradas de una red neuronal deben tener la misma longitud, tendremos que resolver esto más adelante.*"""

len(train_data[0]), len(train_data[1])

"""Volver a convertir los números enteros en palabras

Puede ser útil saber cómo volver a convertir enteros en texto. Aquí, crearemos una función de ayuda para consultar un objeto diccionario que contenga el mapeo de entero a cadena:
"""

word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

"""Utilizar la función decode_review para mostrar el texto de la primera revisión:"""

decode_review(train_data[0])

"""**Dado que las críticas de películas deben tener la misma longitud, utilizar la función pad_sequences para estandarizar las longitudes.**"""

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

len(train_data[0]), len(train_data[1])

"""**Las contribuciones de mayor magnitud tienen un mayor impacto en la predicción del modelo.**"""

print(train_data[0])

"""Modelo propuesto"""

vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
#model.summary()

"""Configurar el modelo para utilizar un optimizador y una función de pérdida:

"""

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

"""Pruebas"""

x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

"""Entrenamiento"""

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

results = model.evaluate(test_data, test_labels)

print(results)

history_dict = history.history
history_dict.keys()

import matplotlib.pyplot as plt

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

